# Net Exposure + PnL Tracker (Underlying ↔ Levered ETFs)

This notebook aggregates **net underlying exposure** by mapping each ETF position into *underlying-equivalent shares*:

\[
\text{Underlying Eq Shares} = \text{ETF Shares} \times \text{Leverage}
\]

It also groups **PnL by underlying**, summing PnL across the underlying stock (if held) plus its associated ETFs.


In [208]:
from pathlib import Path
import pandas as pd
import numpy as np

# --- Inputs (edit if your paths differ) ---
SCREENED_PATH = Path("data/etf_screened_today.csv")  # leverage map & ETF↔underlying map
# Fallbacks if you run this from a different working directory:
FALLBACK_SCREENED = Path("/data/etf_screened_today.csv")

BASE_DIR = Path.cwd().parent          # notebooks/ -> project root
DATA_DIR = BASE_DIR / "data"
# Portfolio positions export (choose ONE source that matches your workflow)
PORT_PATHS = [
    Path(DATA_DIR / "Levered ETF Autotrading.csv")
]
def _first_existing(paths):
    for p in paths:
        if p.exists():
            return p
    return None

screened_path = Path(DATA_DIR / "etf_screened_today.csv")

port_path = _first_existing(PORT_PATHS)
print("Using leverage/map file:", screened_path)
print("Using positions file:", port_path if port_path else "(none found yet)")

Using leverage/map file: c:\Users\werdn\Documents\Investing\ls-algo\data\etf_screened_today.csv
Using positions file: c:\Users\werdn\Documents\Investing\ls-algo\data\Levered ETF Autotrading.csv


## 1) Load ETF→Underlying + Leverage map

`etf_screened_today.csv` should contain at least:
- `ETF`
- `Underlying`
- `Leverage`


In [209]:
screened = pd.read_csv(screened_path)

required = {"ETF", "Underlying", "Leverage"}
missing = required - set(screened.columns)
if missing:
    raise ValueError(f"etf_screened_today.csv missing required columns: {sorted(missing)}")

# clean
screened["ETF"] = screened["ETF"].astype(str).str.strip()
screened["Underlying"] = screened["Underlying"].astype(str).str.strip()
screened["Leverage"] = pd.to_numeric(screened["Leverage"], errors="coerce")
screened = screened.dropna(subset=["ETF", "Underlying", "Leverage"]).copy()

etf_to_under = dict(zip(screened["ETF"], screened["Underlying"]))
etf_to_lev = dict(zip(screened["ETF"], screened["Leverage"]))

screened.head()

,ETF,Underlying,Leverage,cagr_port_hist,LevType,borrow_current,shares_available,borrow_spiking,cagr_positive,borrow_leq_cap,borrow_gt_cap,whitelisted,exclude_borrow_gt_cap,exclude_no_shares,exclude_borrow_spike,include_for_algo
0,AAPU,AAPL,2.0,0.023102,2x,0.049592,40000,False,True,True,False,False,False,False,False,True
1,AMUU,AMD,2.0,0.058359,2x,0.000000,50000,False,True,True,False,False,False,False,False,True
2,AMZU,AMZN,2.0,0.052051,2x,0.058472,60000,False,True,True,False,False,False,False,False,True
3,AVL,AVGO,2.0,0.038841,2x,0.033082,150000,False,True,True,False,False,False,False,False,True
4,BITX,IBIT,2.0,0.063063,2x,0.085008,100000,False,True,True,False,False,False,False,False,True


## 2) Load positions

This notebook is flexible, but it needs at least:

- `symbol` (or `Symbol`)
- `qty` (position size; **short negative**)

Optional but recommended for PnL:
- `avg_cost` (or `avgCost`, `average_cost`)
- `last` (or `price`, `market_price`)
- `unrealized_pnl` (if you already compute it)
- `realized_pnl` (if available)

If your columns differ, edit the `colmap` section.


In [210]:
import numpy as np
import pandas as pd
import re
from pathlib import Path

# ---------- helpers ----------
import numpy as np
import pandas as pd
import re

def parse_number_series(s: pd.Series) -> pd.Series:
    """
    Broker-export safe numeric parser.
    Handles:
      - commas
      - $ signs / text like 'Shares'
      - (123) -> -123
      - blanks / '--' -> NaN
      - non-breaking spaces
    """
    x = s.astype(str)

    # normalize whitespace (including NBSP)
    x = x.str.replace("\u00a0", " ", regex=False).str.strip()

    # treat common null tokens as NaN
    x = x.replace({"": np.nan, "—": np.nan, "–": np.nan, "--": np.nan, "N/A": np.nan, "NA": np.nan})

    # parentheses negatives: (123.45) -> -123.45
    x = x.str.replace(r"^\((.*)\)$", r"-\1", regex=True)

    # remove commas
    x = x.str.replace(",", "", regex=False)

    # remove ANYTHING that's not digit, minus, or dot
    x = x.str.replace(r"[^0-9\.\-]", "", regex=True)

    # collapse multiple minus signs (rare formatting glitches)
    x = x.str.replace(r"^-{2,}", "-", regex=True)

    # empty after stripping -> NaN
    x = x.replace("", np.nan)

    return pd.to_numeric(x, errors="coerce")

def clean_symbol(s: pd.Series) -> pd.Series:
    """
    Clean ticker strings from broker exports.
    Normalizes:
      - trims
      - removes trailing "(EXCH)" if present
      - uppercases
      - converts BRK B / BRK.B -> BRK-B
    """
    x = s.astype(str).str.strip()

    # drop "(NASDAQ)" etc if present
    x = x.str.replace(r"\s*\(.*?\)\s*$", "", regex=True)

    x = s.str.upper()

    # normalize share-class punctuation/spacing: BRK B or BRK.B -> BRK-B
    x = x.str.replace(".", "-", regex=False)
    x = x.str.replace(r"\s+", "-", regex=True)

    return x

# ---------- main loader ----------
def load_positions_from_tws_export(port_path: str | Path) -> pd.DataFrame:
    port_path = Path(port_path)
    if not port_path.exists():
        raise FileNotFoundError(f"Positions file not found: {port_path}")

    pos = pd.read_csv(port_path)
    # Hard-pick these based on your confirmed export
    sym_c = "Financial Instrument"
    qty_c = "Position"

    if sym_c not in pos.columns:
        raise ValueError(f"Expected column '{sym_c}' not found. Columns: {pos.columns.tolist()[:60]}")
    if qty_c not in pos.columns:
        raise ValueError(f"Expected column '{qty_c}' not found. Columns: {pos.columns.tolist()[:60]}")

    df = pos.copy()
    # --- symbol ---
    df["symbol_raw"] = df[sym_c]
    df["symbol"] = clean_symbol(df[sym_c])

    # --- quantity ---
    df["qty"]  =parse_number_series(df[qty_c]).fillna(0.0)
    # Optional columns if present
    if "Avg Price" in df.columns:
        df["avg_cost"] = parse_number_series(df["Avg Price"])
    else:
        df["avg_cost"] = np.nan

    if "Last" in df.columns:
        df["last"] = parse_number_series(df["Last"])
    else:
        df["last"] = np.nan

    if "Unrealized P&L" in df.columns:
        df["unrealized_pnl"] = parse_number_series(df["Unrealized P&L"])
    else:
        df["unrealized_pnl"] = np.nan

    # Realized usually not in this export
    df["realized_pnl"] = 0.0

    # Drop zero positions
    df = df[df["qty"].abs() > 0].copy()

    # Final sanity checks
    df["symbol"] = df["symbol"].astype(str).str.strip()
    df = df[df["symbol"].ne("")].copy()

    # Keep a tidy output schema
    keep = ["symbol", "qty", "avg_cost", "last", "unrealized_pnl", "realized_pnl", "symbol_raw"]
    keep = [c for c in keep if c in df.columns]
    df = df[keep].reset_index(drop=True)

    return df

# --- usage ---
df = load_positions_from_tws_export(port_path)
df


,symbol,qty,avg_cost,last,unrealized_pnl,realized_pnl,symbol_raw
0,BRK-B,70.0,206.724,481.02,19199.0,0.0,BRK B
1,AMD,80.0,244.034,253.35,749.0,0.0,AMD
2,SOLT,-426.0,7.269,6.22,447.0,0.0,SOLT
3,MU,13.0,356.655,394.93,497.0,0.0,MU
4,APPX,-132.0,23.517,22.39,155.0,0.0,APPX
...,...,...,...,...,...,...,...
195,AMDL,-126.0,17.716,21.30,-451.0,0.0,AMDL
196,AMUU,-35.0,61.169,74.09,-474.0,0.0,AMUU
197,DKNG,185.0,33.109,30.38,-509.0,0.0,DKNG
198,IBIT,435.0,52.251,50.26,-875.0,0.0,IBIT


## 3) Classify rows: Underlyings vs ETFs

- If `symbol` exists in `etf_to_under`, treat it as an ETF
- Otherwise, treat it as an underlying stock (direct holding)


In [211]:
import numpy as np
import pandas as pd
import re

def norm_sym(x: str) -> str:
    """Normalize symbols so TWS exports match your mapping keys."""
    if pd.isna(x):
        return ""
    s = str(x).upper().strip()

    # Common normalizations
    s = re.sub(r"\s+", " ", s)        # collapse whitespace
    s = s.replace("/", "-")           # just in case
    s = s.replace(".", "-")           # BRK.B -> BRK-B
    s = s.replace(" ", "-")           # BRK B -> BRK-B

    return s
df["symbol_raw"] = df["symbol"].astype(str)

# take first token before space as ticker
df["symbol"] = df["symbol_raw"].str.upper().str.strip().str.split().str[0]

# --- Build normalized maps (IMPORTANT) ---
etf_to_under_norm = {norm_sym(k): norm_sym(v) for k, v in etf_to_under.items()}
etf_to_lev_norm   = {norm_sym(k): float(v) for k, v in etf_to_lev.items()}

# --- Normalize your df symbols once ---
df["symbol_raw"] = df["symbol"]
df["sym_key"] = df["symbol"].apply(norm_sym)

# --- Mapping ---
df["is_etf"] = df["sym_key"].isin(etf_to_under_norm.keys())

df["underlying"] = np.where(
    df["is_etf"],
    df["sym_key"].map(etf_to_under_norm),
    df["sym_key"],   # stock/fallback = itself (normalized)
)

# leverage: if ETF but leverage missing, default to 1.0 (or raise—your call)
df["leverage"] = np.where(
    df["is_etf"],
    df["sym_key"].map(etf_to_lev_norm).astype(float),
    1.0
)

# Fill any mapping misses defensively
df["underlying"] = df["underlying"].fillna(df["sym_key"])
df["leverage"] = pd.to_numeric(df["leverage"], errors="coerce").fillna(1.0)

# Underlying-equivalent shares contributed by this row
df["under_eq_shares"] = df["qty"] * df["leverage"]

df[["symbol_raw","sym_key","qty","is_etf","underlying","leverage","under_eq_shares"]]


,symbol_raw,sym_key,qty,is_etf,underlying,leverage,under_eq_shares
0,BRK-B,BRK-B,70.0,False,BRK-B,1.0,70.0
1,AMD,AMD,80.0,False,AMD,1.0,80.0
2,SOLT,SOLT,-426.0,True,SOEZ,2.0,-852.0
3,MU,MU,13.0,False,MU,1.0,13.0
4,APPX,APPX,-132.0,True,APP,2.0,-264.0
...,...,...,...,...,...,...,...
195,AMDL,AMDL,-126.0,True,AMD,2.0,-252.0
196,AMUU,AMUU,-35.0,True,AMD,2.0,-70.0
197,DKNG,DKNG,185.0,False,DKNG,1.0,185.0
198,IBIT,IBIT,435.0,False,IBIT,1.0,435.0


In [212]:
df["is_etf"] = df["symbol"].isin(etf_to_under)
df["underlying"] = np.where(df["is_etf"], df["symbol"].map(etf_to_under), df["symbol"])
df["leverage"] = np.where(df["is_etf"], df["symbol"].map(etf_to_lev), 1.0)

# Underlying-equivalent shares contributed by this row
df["under_eq_shares"] = df["qty"] * df["leverage"]

df[["symbol","qty","is_etf","underlying","leverage","under_eq_shares"]]

,symbol,qty,is_etf,underlying,leverage,under_eq_shares
0,BRK-B,70.0,False,BRK-B,1.0,70.0
1,AMD,80.0,False,AMD,1.0,80.0
2,SOLT,-426.0,True,SOEZ,2.0,-852.0
3,MU,13.0,False,MU,1.0,13.0
4,APPX,-132.0,True,APP,2.0,-264.0
...,...,...,...,...,...,...
195,AMDL,-126.0,True,AMD,2.0,-252.0
196,AMUU,-35.0,True,AMD,2.0,-70.0
197,DKNG,185.0,False,DKNG,1.0,185.0
198,IBIT,435.0,False,IBIT,1.0,435.0


In [213]:
df.head(70)

,symbol,qty,avg_cost,last,unrealized_pnl,realized_pnl,symbol_raw,sym_key,is_etf,underlying,leverage,under_eq_shares
0,BRK-B,70.0,206.724,481.02,19199.0,0.0,BRK-B,BRK-B,False,BRK-B,1.0,70.0
1,AMD,80.0,244.034,253.35,749.0,0.0,AMD,AMD,False,AMD,1.0,80.0
2,SOLT,-426.0,7.269,6.22,447.0,0.0,SOLT,SOLT,True,SOEZ,2.0,-852.0
3,MU,13.0,356.655,394.93,497.0,0.0,MU,MU,False,MU,1.0,13.0
4,APPX,-132.0,23.517,22.39,155.0,0.0,APPX,APPX,True,APP,2.0,-264.0
...,...,...,...,...,...,...,...,...,...,...,...,...
65,OPEG,-277.0,10.139,9.81,89.0,0.0,OPEG,OPEG,True,OPEN,2.0,-554.0
66,NVDA,181.0,186.048,187.23,216.0,0.0,NVDA,NVDA,False,NVDA,1.0,181.0
67,XYZY,-125.0,33.416,32.59,93.0,0.0,XYZY,XYZY,True,XYZ,1.0,-125.0
68,ELIL,-99.0,27.767,26.85,62.0,0.0,ELIL,ELIL,True,LLY,2.0,-198.0


## 4) Compute PnL per position (if not already provided)

If `unrealized_pnl` is missing but `avg_cost` and `last` exist, we compute:

\[
UPnL = (last - avg\_cost) \times qty
\]

Total PnL = realized + unrealized


In [214]:
# Fill unrealized_pnl if missing and we have avg_cost & last
need_calc = df["unrealized_pnl"].isna() & df["avg_cost"].notna() & df["last"].notna()
df.loc[need_calc, "unrealized_pnl"] = (df.loc[need_calc, "last"] - df.loc[need_calc, "avg_cost"]) * df.loc[need_calc, "qty"]

# If still missing, keep as 0 for aggregation (but you may want to investigate)
df["unrealized_pnl"] = df["unrealized_pnl"].fillna(0.0)
df["realized_pnl"] = df["realized_pnl"].fillna(0.0)

df["total_pnl"] = df["unrealized_pnl"] + df["realized_pnl"]

df[["symbol","qty","avg_cost","last","unrealized_pnl","realized_pnl","total_pnl"]].head(20)

,symbol,qty,avg_cost,last,unrealized_pnl,realized_pnl,total_pnl
0,BRK-B,70.0,206.724,481.02,19199.0,0.0,19199.0
1,AMD,80.0,244.034,253.35,749.0,0.0,749.0
2,SOLT,-426.0,7.269,6.22,447.0,0.0,447.0
3,MU,13.0,356.655,394.93,497.0,0.0,497.0
4,APPX,-132.0,23.517,22.39,155.0,0.0,155.0
5,ETHT,-77.0,39.231,35.51,290.0,0.0,290.0
6,ASTS,14.0,82.949,108.13,351.0,0.0,351.0
7,HOOW,-95.0,45.275,42.68,234.0,0.0,234.0
8,CEGX,-155.0,20.353,17.75,418.0,0.0,418.0
9,INTW,-48.0,62.176,53.18,435.0,0.0,435.0


## 5) Net exposure per underlying (lever-adjusted)

This is the key view:
- Sum of `under_eq_shares` for all rows mapped to the same underlying.
- Positive = net long underlying-equivalent
- Negative = net short underlying-equivalent


In [215]:
# --- per-leg notional (signed) ---
# assumes:
#   df["qty"]  = Position (signed)
#   df["last"] = Last price (positive)
df = df.copy()

df["mv"] = df["under_eq_shares"] * df["last"]          # signed market value (long +, short -)
df["gross_mv"] = df["mv"].abs()            # gross notional per leg

exposure_by_under = (
    df.groupby("underlying", as_index=False)
      .agg(
          net_notional_usd=("mv", "sum"),
          gross_notional_usd=("gross_mv", "sum"),
          n_legs=("sym_key","nunique"),
          legs=("sym_key", lambda s: ",".join(sorted(set(s))[:20]))
      )
      .sort_values("net_notional_usd", ascending=False)
)

exposure_by_under


,underlying,net_notional_usd,gross_notional_usd,n_legs,legs
13,BRK-B,24030.92,43311.88,3,"BRK-B,BRKC,BRKU"
37,KLAC,452.02,11726.14,2,"KLAC,KLAG"
62,TSLA,382.89,48887.03,6,"TSLA,TSLI,TSLL,TSLR,TSLW,TSLY"
20,COST,358.00,11428.40,2,"COST,COTG"
53,QQQ,323.27,44688.97,6,"JEPQ,QQQ,QQQX,QYLD,QYLG,TQQY"
...,...,...,...,...,...
60,TEM,-250.26,11833.86,2,"TEM,TEMT"
0,AAPL,-263.94,42870.42,5,"AAPB,AAPL,AAPU,AAPX,APLY"
6,APP,-449.36,11372.56,2,"APP,APPX"
43,MSFT,-552.09,43696.41,6,"MSFL,MSFO,MSFT,MSFU,MSFW,MSFX"


In [222]:
u = "CRWV"   # <-- pick any underlying you want to audit

cols = [
    "sym_key",
    "qty",
    "under_eq_shares",
    "last",          # underlying price (confirmed)
    "mv",            # under_eq_shares * last
]

dbg = (
    df.loc[df["underlying"] == u, cols]
      .assign(
          mv_calc=lambda x: x["under_eq_shares"] * x["last"]
      )
      .sort_values("sym_key")
)

print(f"Leg-level exposure for {u}")
display(dbg)

print("\nTotals")
print("Net under_eq_shares :", dbg["under_eq_shares"].sum())
print("Net notional (USD)  :", dbg["mv_calc"].sum())
print("Gross notional (USD):", dbg["mv_calc"].abs().sum())


Leg-level exposure for CRWV


,sym_key,qty,under_eq_shares,last,mv,mv_calc
40,CRWG,-613.0,-1226.0,5.52,-6767.52,-6767.52
30,CRWV,117.0,117.0,105.66,12362.22,12362.22
177,CWVX,-76.0,-152.0,44.58,-6776.16,-6776.16



Totals
Net under_eq_shares : -1261.0
Net notional (USD)  : -1181.46
Gross notional (USD): 25905.899999999998


## 6) PnL by underlying (group of ETFs + underlying)

This groups:
- underlying stock position (if held)
- all associated ETF legs

So you can see which underlying buckets are making/losing money.


In [217]:
EXCLUDE_SYMBOLS = {"BRK-B"}

df = df[~df["symbol"].isin(EXCLUDE_SYMBOLS)].copy()

In [218]:
pnl_by_under = (
    df.groupby("underlying", as_index=False)
      .agg(
          unrealized_pnl=("unrealized_pnl","sum"),
          realized_pnl=("realized_pnl","sum"),
          total_pnl=("total_pnl","sum"),
          net_under_eq_shares=("under_eq_shares","sum"),
      )
      .sort_values("total_pnl", ascending=False)
)

pnl_by_under.head(30)

,underlying,unrealized_pnl,realized_pnl,total_pnl,net_under_eq_shares
46,NFLX,162.0,0.0,162.0,-710.0
13,BRK-B,144.0,0.0,144.0,-340.0
43,MSFT,132.0,0.0,132.0,-860.0
47,NVDA,113.0,0.0,113.0,-1172.0
50,PLTR,109.0,0.0,109.0,-498.0
32,HOOD,92.0,0.0,92.0,-112.0
53,QQQ,70.0,0.0,70.0,-925.0
11,BABA,64.0,0.0,64.0,-340.0
70,XYZ,62.0,0.0,62.0,-63.0
5,AMZN,58.0,0.0,58.0,-754.0


## 7) Drilldown: show the ETF/underlying legs for a chosen underlying

Edit `NAME` and rerun to see the component legs (symbols, qty, leverage, pnl).


In [219]:
NAME = "AAPL"  # <-- change me

drill = df[df["underlying"] == NAME].copy()
drill = drill.sort_values(["is_etf","symbol"])

# (optional) make sure each leg has its own notional too
if "net_notional_usd" not in drill.columns:
    drill["leg_notional_usd"] = drill["mv"]

# --- JOIN: add underlying totals onto every leg row ---
drill_joined = drill.merge(
    exposure_by_under[["underlying", "net_notional_usd", "gross_notional_usd", "n_legs"]],
    on="underlying",
    how="left",
    suffixes=("", "_under")
)

drill_joined[[
    "symbol", "qty", "leverage", "under_eq_shares", "last",
    "mv",                       # leg signed notional
    "net_notional_usd",         # underlying net notional (same for every leg)
    "gross_notional_usd",
    "unrealized_pnl", "realized_pnl", "total_pnl"
]]

,symbol,qty,leverage,under_eq_shares,last,mv,net_notional_usd,gross_notional_usd,unrealized_pnl,realized_pnl,total_pnl
0,AAPL,84.0,1.0,84.0,253.61,21303.24,-263.94,42870.42,156.0,0.0,156.0
1,AAPB,-105.0,2.0,-210.0,27.49,-5772.90,-263.94,42870.42,-34.0,0.0,-34.0
2,AAPU,-100.0,2.0,-200.0,28.95,-5790.00,-263.94,42870.42,20.0,0.0,20.0
3,AAPX,-110.0,2.0,-220.0,26.21,-5766.20,-263.94,42870.42,-122.0,0.0,-122.0
4,APLY,-344.0,1.0,-344.0,12.32,-4238.08,-263.94,42870.42,28.0,0.0,28.0


## 8) Portfolio totals + exports

Writes two CSVs you can use for dashboards / daily emails:
- `net_exposure_by_underlying.csv`
- `pnl_by_underlying.csv`


In [220]:
totals = {
    "total_unrealized_pnl": float(df["unrealized_pnl"].sum()),
    "total_realized_pnl": float(df["realized_pnl"].sum()),
    "total_pnl": float(df["total_pnl"].sum()),
}
totals

{'total_unrealized_pnl': -382.0,
 'total_realized_pnl': 0.0,
 'total_pnl': -382.0}

In [221]:
out_dir = Path("data") / "reports"
out_dir.mkdir(parents=True, exist_ok=True)

exposure_csv = out_dir / "net_exposure_by_underlying.csv"
pnl_csv = out_dir / "pnl_by_underlying.csv"

exposure_by_under.to_csv(exposure_csv, index=False)
pnl_by_under.to_csv(pnl_csv, index=False)

print("Wrote:", exposure_csv)
print("Wrote:", pnl_csv)

Wrote: data\reports\net_exposure_by_underlying.csv
Wrote: data\reports\pnl_by_underlying.csv
